# Likelyhood of winning a championship based on car's performance history


The main objective of this analysis is to measure how much impact the deviation on car's performance determine championship's results.

The approach will be identify how much in seconds each car is faster based on all races and then select one championship to predict what would be the result if there were no performance diferences between cars.

The data was obtained scraping [KGV's website](http://kartodromogranjaviana.com.br/), one of the biggest kart circuits in Latin America, based on Granja Viana - Sao Paulo.

At KGV kart circuit there are several car categories that are allowed to performe a race. To that analysis, we are going to focus only on races in which rented cars were used. The reason is that by following this approach we have a better chance to correctly identify and link cars between diferent races. Furthermore, the data of rented races is much richier than other categories.

To tackle our objective, we are going to based our analysis on KGV's Granja Viana circuit in 2019, resticted to rented car's races.

We are going to use the following assumptions:
* Car's numbers doesn't change between the actual cars (confirmed by kart circuit's adms)
* Championship points scoring system is equal to [F1 in 2019](https://en.wikipedia.org/wiki/List_of_Formula_One_World_Championship_points_scoring_systems)

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
kart_data = pd.read_csv("../Data/kart_data.csv")
kart_data.describe()

,No.,na volta.,Voltas
count,580.000000,580.000000,129.000000
mean,22.975862,12.065517,20.837209
std,15.205311,5.506082,6.545264
min,1.000000,0.000000,1.000000
25%,10.000000,9.000000,14.000000
50%,22.000000,12.000000,25.000000
75%,33.000000,15.000000,26.000000
max,80.000000,34.000000,28.000000


In [ ]:
kart_data.head()

In [ ]:
kart_data.groupby('No.').count()['Pos'].describe()

### Selecting just karts with 100 or more races

In [ ]:
filter_karts_500races = kart_data.groupby('No.').count()['Pos'] >= 500
karts_500races = kart_data.groupby('No.').count()['Pos'][filter_karts_500races].reset_index()["No."]
karts_500races.head()

In [ ]:
filter_karts = kart_data['No.'].isin(karts_500races)
kart_data = kart_data[filter_karts]
kart_data.describe()

In [ ]:
kart_data['Pos'] = pd.to_numeric(kart_data['Pos'],errors='coerce')
kart_data.pivot(columns='No.',values='Pos').describe()

In [ ]:
# the boxplot
fig1, ax1 = plt.subplots()
data=[]
for i in range(1,20):
    _data = kart_data.loc[(kart_data['No.']==i) & (kart_data['Pos'] > 0), 'Pos']
    _data_mean = np.mean(_data)
    data.append((i,_data_mean,data))

data.sort(key=lambda x: x[1])

# ax1.boxplot(data[0][2], vert=False, meanline=True)

In [ ]:
kart_data.loc[kart_data['No.']==1]['Pos'].reset_index().head()

In [ ]:
# Fixing random state for reproducibility
np.random.seed(19680801)

# fake up some data
spread = np.random.rand(50) * 100
center = np.ones(25) * 50
flier_high = np.random.rand(10) * 100 + 100
flier_low = np.random.rand(10) * -100
data = np.concatenate((spread, center, flier_high, flier_low))

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_title('Basic Plot')
ax1.boxplot(data)

In [ ]:

data